### Stratégie pour Utiliser le fichie JSON avec GPT-4o en RAG
	1.	Indexer les questions et réponses dans une base vectorielle (FAISS, ChromaDB).
	2.	Lorsqu’un utilisateur pose une question, récupérer la réponse la plus pertinente.
	3.	Envoyer la réponse trouvée + la question originale à GPT-4o pour générer une réponse contextualisée.

### Préparer la Base de Données Vectorielle

On va transformer les instructions et inputs en vecteurs pour effectuer des recherches de similarité.

In [9]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Charger les données du fichier JSON
with open("/Users/moussa-kalla/Datathon/data/faq.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extraire les instructions et inputs comme données à indexer
texts = [entry["instruction"] + " " + entry["input"] for entry in data]
responses = [entry["output"] for entry in data]

# Utiliser SentenceTransformer pour encoder les textes
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(texts)

# Créer l'index FAISS pour la recherche rapide
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Sauvegarder l'index et les réponses
faiss.write_index(index, "/Users/moussa-kalla/Datathon/data/faq.faiss")
with open("/Users/moussa-kalla/Datathon/data/responses.json", "w", encoding="utf-8") as f:
    json.dump(responses, f)

print("Base de données vectorielle créée ✅")

Base de données vectorielle créée ✅


### Rechercher la Réponse la Plus Pertinente

Quand un utilisateur pose une question, on va chercher la réponse la plus proche dans FAISS.

In [12]:
def search_faq(query):
    query_vector = model.encode([query])  # Encoder la question

    # Charger l'index et les réponses
    index = faiss.read_index("/Users/moussa-kalla/Datathon/data/faq.faiss")
    with open("/Users/moussa-kalla/Datathon/data/responses.json", "r", encoding="utf-8") as f:
        responses = json.load(f)

    # Trouver la réponse la plus proche
    _, idx = index.search(np.array(query_vector), k=1)  
    return responses[idx[0][0]]  # Retourner la meilleure réponse

# Exemple de recherche
question = "Comment nettoyer mon fauteuil roulant électrique ?"
print(search_faq(question))

Essuyez les pièces à l'aide d'un chiffon ou d'une brosse souple, d'agents de nettoyage ménagers ordinaires et d'eau chaude. Rincez ensuite à l'eau chaude et essuyez soigneusement les pièces avec un chiffon sec.


### Intégrer GPT-4o avec OpenAI

Une fois qu’on a récupéré la réponse la plus pertinente, on l’envoie à GPT-4o en contexte pour une réponse plus fluide.

In [ ]:
from openai import OpenAI

client = OpenAI()

def ask_gpt4o(question):
    retrieved_answer = search_faq(question)  # Cherche la meilleure réponse

    # Construire le prompt avec le contexte
    prompt = f"Voici une réponse issue de la base de connaissances :\n{retrieved_answer}\n\nQuestion : {question}\nRéponds avec précision en tenant compte du contexte."

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000
    )

    return response.choices[0].message.content

# Exemple d'utilisation
question = "Quels sont les modèles de fauteuil roulant électrique storm ?"
print(ask_gpt4o(question))

Les modèles de fauteuils roulants électriques de la série Storm incluent généralement plusieurs variantes qui sont adaptées à différents besoins des utilisateurs. Parmi ces modèles, on trouve souvent :

1. Storm4: Connu pour sa facilité d'entretien, comme mentionné dans votre extrait, avec un accès simplifié aux composants électroniques.
2. Storm4 X-plore: Une version du Storm4 qui est souvent équipée de roues et de suspensions adaptées pour un usage en extérieur et sur terrains accidentés.
3. Storm3: Un ancien modèle qui était également populaire pour sa fiabilité et ses fonctionnalités.

Chaque modèle peut proposer différentes configurations et options pour s'adapter aux besoins spécifiques des utilisateurs, comme des systèmes de positionnement personnalisés, différents types de commandes ou des batteries de longue durée.
